In [1]:
#code written by Sara Irvine with help from Trygve Mangus Reader

In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import scipy.ndimage
import skimage.measure
import plotly.graph_objects as go
import matplotlib
from plotly.offline import plot
import plotly.graph_objs as go
from google.colab import drive#if you want to run it from google collab(ideally don't run from google collab)
drive.mount('/content/drive')




with open('/content/drive/MyDrive/EuXFEL 3052/CODE/3D strucutre/WeakBeamMinus3Dstructurescan.pkl', 'rb') as f:
    arr = np.array(pickle.load(f))

 #array of z postive from the motor positions it was easier to just type it in rathen than load it in every time
zposition=[-31.0571, -31.0529, -31.0488, -31.0447,
 -31.0405, -31.0365, -31.0324, -31.0283, -31.0242,
 -31.0200, -31.0159, -31.0118, -31.0077, -31.0036,
 -30.9995, -30.9954, -30.9913, -30.9872, -30.9831,
 -30.9790, -30.9749, -30.9708, -30.9667,-30.9626, -30.9585, -30.9544, -30.9503, -30.9462,-30.9421, -30.9380, -30.9339, -30.9298,
 -30.9257, -30.9216, -30.9175, -30.9134, -30.9093,
 -30.9052, -30.9011, -30.8970]


#cropping out the surface as it is harder to work with
arr2 = np.copy(arr[:,500:,:])


# Perform binary erosion on 'arr2'
structure = np.ones((3,10, 10), dtype=np.int64)

eroded_arr = scipy.ndimage.binary_erosion(arr2, structure=structure).astype(np.int64)
binary_erosion_mask = (eroded_arr == 1)

# Perform binary dilation on the eroded data
dilated_arr = scipy.ndimage.binary_dilation(binary_erosion_mask, structure=structure).astype(np.int64)

# Replace the dilated values in 'binned_arr' with their corresponding values from 'arr2'
result_data = np.where(dilated_arr, arr2, eroded_arr)

#iterate for 5 cycles
i=1

while i<5:
    eroded_arr = scipy.ndimage.binary_erosion(result_data, structure=structure).astype(np.int64)
    binary_erosion_mask = (eroded_arr == 1)

    # Perform binary dilation on the eroded data
    dilated_arr = scipy.ndimage.binary_dilation(binary_erosion_mask, structure=structure).astype(np.int64)

    # Replace the dilated values in 'binned_arr' with their corresponding values from 'arr2'
    result_data = np.where(dilated_arr, result_data, eroded_arr)

    i=i+1


# Apply binning to the eroded data
scale = 10 # Binning scale
s = result_data.shape
binned_eroded_arr = result_data.reshape((-1, s[1] // scale, scale, s[2] // scale, scale)).sum(4).sum(2) / scale / scale
print(binned_eroded_arr.shape)

#median filtering after binning(tested different filtering and decied on the median filter rather than a weaker or stronger one)
footprint = np.ones((2, 4, 4))  # Define the size of the neighborhood
binned_eroded_arr= scipy.ndimage.median_filter(binned_eroded_arr , footprint=footprint)




threshold = 0.8

#  Create a mask where values are below the threshold
mask = binned_eroded_arr< threshold

# Set values below the threshold to NaN
binned_eroded_arr[mask] = np.nan


#test the filtering vs the unfiltered data for a signle z position
fig, axes = plt.subplots(1,2, figsize = (12,6))
axes[0].imshow(binned_eroded_arr[10], vmin = 0, vmax = 1.5)
axes[1].imshow(arr2[10], vmin = 0, vmax = 1.5)

#setting up the marching cubes(mcs) plotting

mcs = []
thresholds = [0.8,0.9,1,1.1,1.2]
for val in thresholds:
    mcs.append(skimage.measure.marching_cubes(binned_eroded_arr, val))


viridis = matplotlib.cm.get_cmap('Greys')







In [ ]:


#3d ploting

data = []
for index, mc in enumerate(mcs):
    verts, faces, normals, values = mc
    z, y, x = verts.T

    i, j, k = np.asarray(faces).T
    color = viridis(index / (len(mcs) - 1))
    color = (color[0], color[1], color[2], 0.2)

    #0.0369 is the pixels to um
    #4 is the um per step size


    # Define scaling factors for x, y, and z
    x_scale = 0.0369*scale/(np.sin(np.deg2rad(35)))
    y_scale = 0.0369*scale
    z_scale = 4/(np.cos(np.deg2rad(2.3)) )

    # Apply scaling to x, y, and z
    x_scaled = x * x_scale
    y_scaled = y * y_scale
    z_scaled = z*z_scale    # Create a numpy array with custom z values

    data.append(go.Mesh3d(x= x_scaled, y= y_scaled, z= z_scaled,
                           i=i, j=j, k=k,
                           color=f'rgba{color}')
                )

fig = go.Figure(data=data)


fig.update_layout(

    paper_bgcolor="white",
    scene=dict(
     xaxis=dict(gridcolor="black"),  # Set the x-axis grid color to black
     yaxis=dict(gridcolor="black"),  # Set the y-axis grid color to black
     zaxis=dict(gridcolor="black"),
     xaxis_title='Y (µm)',
    yaxis_title='Z (µm)',
    zaxis_title='X (µm)',
                        )# Set the z-axis grid color to black

 ),



fig.show()

In [ ]:
#rotation movie ploting
import plotly.graph_objects as go
import numpy as np



x_eye = -2.25
y_eye = 3
z_eye = 1.5

fig.update_layout(
         title='Animation Test',
         width=600,
         height=600,
         scene_camera_eye=dict(x=x_eye, y=y_eye, z=z_eye),
         scene=dict(
                xaxis_title='Y (µm)',
                yaxis_title='Z (µm)',
                zaxis_title='X (µm)',
                        ),
         updatemenus=[dict(type='buttons',
                  showactive=False,
                  y=1,
                  x=0.8,
                  xanchor='left',
                  yanchor='bottom',
                  pad=dict(t=45, r=10),
                  buttons=[dict(label='Play',
                                 method='animate',
                                 args=[None, dict(frame=dict(duration=5, redraw=True),
                                                             transition=dict(duration=0),
                                                             fromcurrent=True,
                                                             mode='immediate'
                                                            )]
                                            )
                                      ]
                              )
                        ]
)


def rotate_z(x, y, z, theta):
    w = x+1j*y
    return np.real(np.exp(1j*theta)*w), np.imag(np.exp(1j*theta)*w), z

frames=[]
for t in np.arange(0, 6.26, 0.1):
    xe, ye, ze = rotate_z(x_eye, y_eye, z_eye, -t)
    frames.append(go.Frame(layout=dict(scene_camera_eye=dict(x=xe, y=ye, z=ze))))
fig.frames=frames

fig.show()